# AutoPrompt & OPRO: Automated Gradient‑Based Prompt Engineering
This notebook walks through two state‑of‑the‑art automated prompt discovery methods:
1. **AutoPrompt** – greedily inserts tokens that maximise a supervised objective.
2. **OPRO (One‑Prompt Reasoning Optimisation)** – searches for chain‑of‑thought prompts that maximise *reasoning* reward.

*Pedagogical goal:* Show how **model gradients** and **search/optimisation** can find high‑performing prompts without human intuition.

## 1. Install deps

In [ ]:
!pip -q install --upgrade transformers datasets trl einops accelerate


## 2. AutoPrompt on Sentiment
We use the original sentiment‑analysis task from the paper.

In [ ]:
import torch, itertools, random, re, math
from datasets import load_dataset
from transformers import AutoModelForMaskedLM, AutoTokenizer

model_name = 'bert-base-uncased'
tok = AutoTokenizer.from_pretrained(model_name)
mlm = AutoModelForMaskedLM.from_pretrained(model_name)
dataset = load_dataset('sst2', split='train[:100]')


### Helper: compute accuracy of a prompt template
Template: `[CLS] {prompt} {sentence} It was [MASK]. [SEP]`

In [ ]:
def prompt_accuracy(prompt_tokens):
    correct=0
    for ex in dataset:
        sent=ex['sentence']
        label=ex['label']
        text = tok.cls_token + ' ' + ' '.join(prompt_tokens) + ' ' + sent + ' It was ' + tok.mask_token + '.'
        inp = tok(text, return_tensors='pt')
        with torch.no_grad():
            logits = mlm(**inp).logits
        mask_ix = (inp['input_ids'][0]==tok.mask_token_id).nonzero()[0]
        probs = logits[0, mask_ix].softmax(-1)
        pred = probs[tok.convert_tokens_to_ids('great')] > probs[tok.convert_tokens_to_ids('terrible')]
        if pred.item() == label: correct +=1
    return correct/len(dataset)


### AutoPrompt loop (simplified – one position at a time)

In [ ]:
vocab=list(tok.get_vocab().keys())
prompt=['it']*3  # 3‑token prompt stub
for pos in range(len(prompt)):
    best=(None,0)
    subset=random.sample(vocab,500)  # small subset for speed
    for w in subset:
        trial=prompt.copy(); trial[pos]=w
        acc=prompt_accuracy(trial)
        if acc>best[1]: best=(w,acc)
    prompt[pos]=best[0]
    print(f'Pos {pos} -> {best}')
print('Auto‑discovered prompt:', prompt)
print('Final acc:', prompt_accuracy(prompt))

## 3. OPRO for arithmetic chain‑of‑thought
OPRO uses a **reward model** (here: correctness of arithmetic) and **search over prompts**.
*We implement a tiny evolutionary hill‑climb for brevity.*

In [ ]:
from transformers import AutoModelForCausalLM
lm=AutoModelForCausalLM.from_pretrained('gpt2')

def evaluate(prompt):
    q='What is 13 plus 24?'
    full=prompt+q
    out=tok(full, return_tensors='pt').to(lm.device)
    gen=lm.generate(**out, max_new_tokens=10)
    ans=tok.decode(gen[0], skip_special_tokens=True).split('\n')[-1]
    return abs(int(re.findall(r'\d+', ans)[0]) - 37)  # reward = distance to correct

population=['Let us solve step by step: 13 + 24 =',
           'First compute 13+24 to get',
           'Answer to 13 plus 24 is']
for gen in range(5):
    scored=[(p,evaluate(p)) for p in population]
    scored.sort(key=lambda x:x[1])
    print(f'Gen {gen} best:', scored[0])
    # mutate top 2
    parents=[p for p,_ in scored[:2]]
    population=parents+ [p+' therefore' for p in parents]


### Reflection
- **AutoPrompt** reveals *heuristic tokens* that implicitly steer a frozen LM.
- **OPRO** treats prompt text as a policy optimised by search.
- Both require *no gradient updates to model weights*, yet achieve surprising gains.